In [111]:
#import NumPy and Random libraries
import numpy as np
import random


#Randomly generate a 4-bit binary message
def message():
    msg = []
    #randomly select either a zero or one until there are 4 values appended to the msg list
    for i in range(4):
        letter = random.choice([0,1])
        msg.append(letter)
    return np.array(msg)

#Check that a custom message is binary and 4-bits
def custom_message_check(msg):
    binary=[0,1]
    for i in msg:
        #confirm that each digit is either a one or zero, and that the entire message is no more than 4 digits 
        if i in binary and len(msg)==4:
            continue
        else:
            raise ValueError("Please enter a 4-bit binary word.")
    return np.array(msg)



#Hamming's Generator Matrix (7x4)
G =  np.array([[1, 1, 0, 1],[1, 0, 1, 1],[1, 0, 0, 0],[0, 1, 1, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

#Hamming's Parity-check Matrix (3x7)
H = np.array([[1, 0, 1, 0, 1, 0, 1],[0, 1, 1, 0, 0, 1, 1],[0, 0, 0, 1, 1, 1, 1]])

#Hamming's Decoder Matrix (4x7)
R = np.array([[0,0,1,0,0,0,0], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])



#Encode the message using Generator Matrix
#done by multiplying G and the message to get the dot products, this returns the 7-bit codeword
def encode(msg):
    enc = np.dot(G, msg)%2  #modulo of 2, to keep code binary
    return enc

#Check for errors in message after it has been transmitted (gone through the noisy channel) using Parity Check Matrix
#done by multiplying H and the codeword to get the dot products, and returns a 3-bit syndrome vector
def paritycheck(noisy):
    par = np.dot(H, noisy)%2
    return par

#Decode the corrected (if necessary to correct) message back to original 4-bits using Decoder Matrix
def decode(corrected):
    dec = np.dot(R, corrected)%2
    return dec



#Simulate noisy channel by flipping number of bits requested when running function
#bits = random.choice([0,1,2]) would be an alternative route to automate process, 
#but would not allow for deliberate testing and testing of errors outside of scope of Hamming code
def noise(enc, bits):
    
    noisy = np.copy(enc)
    #unpack the noisy array to get a list of all indexes (which will serve as the population for the sample)
    index=[*range(0,len(noisy),1)]
    
    #confirm that no more than three bits of error are being introduced (which is unlikely realistically)
    possible_bits=[0,1,2,3]
    if bits in possible_bits:
        #use random.sample to randomly select the number of indexes for bits requested to be flipped
        e = random.sample(index, bits)
    else:
        raise ValueError("Please only request between 0 and 3 bits of error to be introducted.")
    
    #iterate through sample and flip the bits for the indexes selected
    for i in e:
        if noisy[i] == 0:
            noisy[i] = 1
        else:
            noisy[i] = 0

    return noisy


#generate error vector
def find_error(par):
    #transpose H so as to group values in each column together
    #creates a "library" of 3-bit arrays to compare the syndrome vector to
    H_trans= H.transpose()
    error_vector=[]
    #iterate through arrays in the matrix and compare them to the syndrome vector
    #create a list by appending 0's where the arrays do not match, and a 1 where they do. 
    for i in H_trans:
        if np.array_equal(i,par):
            error_vector.append(1)
        else:
            error_vector.append(0)

    error_vector=np.array(error_vector)
    return error_vector


#add error vector to noisy vector in order to make the correction 
#by "flipping" the one bit that had previously been flipped back to it's original state
def correct(noisy, error_vector):
    corrected=np.add(noisy,error_vector)%2
    return corrected



#All above functions combined for complete Hamming function with a custom 4-bit binary word to transmit
def customHamming(msg,bits):
    
    msg = custom_message_check(msg)
    print ("4-bit message: ", msg)
    enc = encode(msg)
    print ("Encoded message: ", enc)
    noisy = noise(enc, bits)
    print ("Noisy message: ", noisy)
    par = paritycheck(noisy)
    print ("Syndrome vector: ", par)
    error_vec = find_error(par)
    print ("Error vector: ", error_vec)
    corrected = correct(noisy, error_vec)
    print ("Correction attempt: ", corrected)
    dec = decode(corrected)
    print ("Decoded message: ", dec)
    
    #confirm that the original & decoded message match, and that an error was detected via the parity check 
    if np.array_equal(msg,dec) and np.array_equal(par,[0,0,0])==False:
        return "One error has been detected and has been corrected."
    
    #confirm that the original & decoded message DO NOT match, but that an error was detected via the parity check 
    elif np.array_equal(msg,dec)==False and np.array_equal(par,[0,0,0])==False:
        return "An error has been detected, but could not be corrected. Please remember, Hamming code is only able to correct one bit errors, and identify one and two bit errors."
    
    #in case of no error detected via the parity check
    else:
        return "No error detected, but please remember an error may still be present. Hamming code is only able to detect one and two bit errors."
        

#All above functions combined for complete Hamming function with a randomly generated 4-bit word to transmit
def autoHamming(bits):
    
    msg = message()
    print ("4-bit message: ", msg)
    enc = encode(msg)
    print ("Encoded message: ", enc)
    noisy = noise(enc, bits)
    print ("Noisy message: ", noisy)
    par = paritycheck(noisy)
    print ("Syndrome vector: ", par)
    error_vec = find_error(par)
    print ("Error vector: ", error_vec)
    corrected = correct(noisy, error_vec)
    print ("Correction attempt: ", corrected)
    dec = decode(corrected)
    print ("Decoded message: ", dec)
    
    #confirm that the original & decoded message match, and that an error was detected via the parity check 
    if np.array_equal(msg,dec) and np.array_equal(par,[0,0,0])==False:
        return "One error has been detected and has been corrected."
    
    #confirm that the original & decoded message DO NOT match, but that an error was detected via the parity check    
    elif np.array_equal(msg,dec)==False and np.array_equal(par,[0,0,0])==False:
        return "An error has been detected, but could not be corrected. Please remember, Hamming code is only able to correct one bit errors, and identify one and two bit errors."
    
    #in case of no error detected via the parity check
    else:
        return "No error detected, but please remember an error may still be present. Hamming code is only able to detect one and two bit errors."
        
    
#Testing of successful correction of 1 bit errors with custom message entered
def MessageCorrection(msg, runs):
    trial=0
    count=0
    bits=1 
    
    #run the trial the requested number of times
    while trial<runs:
        msg = custom_message_check(msg)
        enc = encode(msg)
        noisy = noise(enc, bits)
        par = paritycheck(noisy)
        error_vec = find_error(par)
        corrected = correct(noisy, error_vec)
        dec = decode(corrected)

        #confirm that the original & decoded message match, and that an error was detected via the parity check 
        if np.array_equal(msg,dec) and np.array_equal(par,[0,0,0])==False:
            count+=1
            trial+=1
        #in case the original & decoded message DO NOT match
        else:
            trial+=1
            continue
        
    percentage_correct= (count/runs)*100
    return print("Out of ", runs," runs, Hamming code identified and corrected a one bit error", percentage_correct, " percent of the time.")
                
    
#Testing of successful identification (should be 100% from 1 and 2 bit errors)
def ErrorIdentify(bits, runs):
    trial=0
    count=0
    
    while trial<runs:
        msg = message()
        enc = encode(msg)
        noisy = noise(enc, bits)
        par = paritycheck(noisy)
        
        #confirm that an error was detected via the parity check 
        if np.array_equal(par,[0,0,0])==False:
            count+=1
            trial+=1
        #in case an error was not detected via the parity check
        else:
            trial+=1
            continue
            
    percentage_correct= (count/runs)*100
    return print("Out of ", runs," runs, Hamming code identified the presence of an error ", percentage_correct, " percent of the time.")
            


### Test #1: Custom Message Testing - enter a 4-bit binary message (as a list) and how many errors you would like introduced through the "noisy channel".

In [112]:
customHamming([1,0,1,0],1)

4-bit message:  [1 0 1 0]
Encoded message:  [1 0 1 1 0 1 0]
Noisy message:  [1 0 0 1 0 1 0]
Syndrome vector:  [1 1 0]
Error vector:  [0 0 1 0 0 0 0]
Correction attempt:  [1 0 1 1 0 1 0]
Decoded message:  [1 0 1 0]


'One error has been detected and has been corrected.'

In [113]:
customHamming([1,0,1,10],1)

ValueError: Please enter a 4-bit binary word.

### Test 2: Custom message one-bit error correcting capability - enter a 4-bit binary message (as a list) and how many tests to perform.

In [114]:
MessageCorrection([1,0,1,0],10000)

Out of  10000  runs, Hamming code identified and corrected a one bit error 100.0  percent of the time.


### Test #3: Random message generated - Test by changing the input number to how many errors you want introduced through the "noisy channel". 

In [115]:
Hamming(0)

4-bit message:  [1 1 1 1]
Encoded message:  [1 1 1 1 1 1 1]
Noisy message:  [1 1 1 1 1 1 1]
Syndrome vector:  [0 0 0]
Error vector:  [0 0 0 0 0 0 0]
Correction attempt:  [1 1 1 1 1 1 1]
Decoded message:  [1 1 1 1]


'No error detected, but please remember an error may still be present. Hamming code is only able to detect one and two bit errors.'

In [116]:
Hamming(1)

4-bit message:  [0 0 0 1]
Encoded message:  [1 1 0 1 0 0 1]
Noisy message:  [0 1 0 1 0 0 1]
Syndrome vector:  [1 0 0]
Error vector:  [1 0 0 0 0 0 0]
Correction attempt:  [1 1 0 1 0 0 1]
Decoded message:  [0 0 0 1]


'One error has been detented and has been corrected.'

In [117]:
Hamming(2)

4-bit message:  [0 1 1 1]
Encoded message:  [0 0 0 1 1 1 1]
Noisy message:  [0 1 0 1 1 0 1]
Syndrome vector:  [0 0 1]
Error vector:  [0 0 0 1 0 0 0]
Correction attempt:  [0 1 0 0 1 0 1]
Decoded message:  [0 1 0 1]


'An error has been detected, but could not be corrected. Please remember, Hamming code is only able to correct one bit errors, and identify one and two bit errors.'

In [118]:
Hamming(3)

4-bit message:  [1 0 1 1]
Encoded message:  [0 1 1 0 0 1 1]
Noisy message:  [1 1 1 0 1 1 0]
Syndrome vector:  [1 1 0]
Error vector:  [0 0 1 0 0 0 0]
Correction attempt:  [1 1 0 0 1 1 0]
Decoded message:  [0 1 1 0]


'An error has been detected, but could not be corrected. Please remember, Hamming code is only able to correct one bit errors, and identify one and two bit errors.'

### Test #4: Test by entering how many errors you would like to introduce (how many bits to flip) and how many trials you would like to run. Hamming code should identify 100% of 1 and 2 bit errors. 

In [119]:
ErrorIdentify(0,10000)

Out of  10000  runs, Hamming code identified the presence of an error  0.0  percent of the time.


In [120]:
ErrorIdentify(1,10000)

Out of  10000  runs, Hamming code identified the presence of an error  100.0  percent of the time.


In [121]:
ErrorIdentify(2,10000)

Out of  10000  runs, Hamming code identified the presence of an error  100.0  percent of the time.


In [122]:
ErrorIdentify(3,10000)

Out of  10000  runs, Hamming code identified the presence of an error  80.15  percent of the time.
